In [40]:
from scipy.io import arff
import pandas as pd

data_aloi, meta_aloi = arff.loadarff('ALOI/ALOI/ALOI.arff')
data_glass, meta_glass = arff.loadarff('Glass/Glass/glass.arff')
data_ionos, meta_ionos = arff.loadarff('Ionosphere/Ionosphere/ionos.arff')
# Convert to DataFrame
df_aloi = pd.DataFrame(data_aloi)
df_glass = pd.DataFrame(data_glass)
df_ionos = pd.DataFrame(data_ionos)


In [3]:
# préparation du dataset et des label
# suppression des label et nettoyage des labels avec 1 et 0
from sklearn.preprocessing import LabelEncoder

def prepare_dataset(df, col_name):
    y_aloi = df.copy()
    # Supprimer la colonne 'outlier' du DataFrame
    df_aloi = df.drop(columns=col_name)
    label_encoder = LabelEncoder()
    y_aloi[col_name] = label_encoder.fit_transform(y_aloi[col_name])
    return df_aloi, y_aloi

In [4]:
# calculs des outliers avec LOF selon les paramètes
# return un tableau des outliers detectés
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import train_test_split

def compute_outliers_LOF(df, n_neig, cont): 
    lof_model = LocalOutlierFactor(n_neighbors=n_neig, contamination=cont)
    outlier_scores = lof_model.fit_predict(df)
    # -1 outlier et 1 inlier
    # convertion en 1 outlier et 0 inlier
    outliers_detected = outlier_scores == -1
    return outliers_detected.astype(int)

In [43]:
# application de LOF
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import ParameterGrid
def run_search_best_params(df, y, label_col):    
    # Paramètres à tester
    param_grid = {
        'n_neighbors': [3, 5, 10],
        'contamination': [0.005, 0.01, 0.05, 0.1]
    }
    
    # Meilleurs résultats initiaux
    meilleur_f1 = 0
    meilleurs_params = None
    
    # combinaisons de paramètres
    for params in ParameterGrid(param_grid):
        outliers_detected = compute_outliers_LOF(df, params['n_neighbors'], params['contamination'])
        
        f1 = f1_score(y[label_col], outliers_detected)
        precision = precision_score(y[label_col], outliers_detected)
        recall = recall_score(y[label_col], outliers_detected)

        print(f"Paramètres : {params}")
        print(f"Précision : {precision * 100:.2f}%")
        print(f"Recall : {recall * 100:.2f}%")
        print(f"F1 score : {f1:.2f}")
        print("----------------------")
        
        # Mettre à jour les meilleurs paramètres si nécessaire
        if f1 > meilleur_f1:
            meilleur_f1 = f1
            meilleurs_params = params
    
    print("Meilleurs paramètres:")
    print(meilleurs_params)
    print("Meilleur F1 score : {:.2f}".format(meilleur_f1))

In [44]:
# test pour dataset ALOI - 
# ALOI 49534 objects, 1508 outliers (3.04%) ALOI
print("computing ALOI dataset...")
df_aloi, y_aloi = prepare_dataset(df_aloi, 'outlier')
run_search_best_params(df_aloi, y_aloi, 'outlier')

# test pour dataset glass
# 7 attributes, 214 objects, 9 outliers (4.21%)
print("computing GLASS dataset...")
df_glass, y_glass = prepare_dataset(df_glass, 'outlier')
run_search_best_params(df_glass, y_glass, 'outlier')


# test pour dataset Ionosphere
# 32 attributes, 351 objects, 126 outliers (35.90%)
print("computing IONOS dataset...")
df_ionos, y_ionos = prepare_dataset(df_ionos, 'outlier')
run_search_best_params(df_ionos, y_ionos, 'outlier')


computing ALOI dataset...
Paramètres : {'contamination': 0.005, 'n_neighbors': 3}
Précision : 49.19%
Recall : 8.09%
F1 score : 0.14
----------------------
Paramètres : {'contamination': 0.005, 'n_neighbors': 5}
Précision : 47.98%
Recall : 7.89%
F1 score : 0.14
----------------------
Paramètres : {'contamination': 0.005, 'n_neighbors': 10}
Précision : 2.42%
Recall : 0.40%
F1 score : 0.01
----------------------
Paramètres : {'contamination': 0.01, 'n_neighbors': 3}
Précision : 37.10%
Recall : 12.20%
F1 score : 0.18
----------------------
Paramètres : {'contamination': 0.01, 'n_neighbors': 5}
Précision : 38.51%
Recall : 12.67%
F1 score : 0.19
----------------------
Paramètres : {'contamination': 0.01, 'n_neighbors': 10}
Précision : 1.21%
Recall : 0.40%
F1 score : 0.01
----------------------
Paramètres : {'contamination': 0.05, 'n_neighbors': 3}
Précision : 18.41%
Recall : 30.24%
F1 score : 0.23
----------------------
Paramètres : {'contamination': 0.05, 'n_neighbors': 5}
Précision : 20.55

# Explication des données
- Le dataset ALOI contient des images d'objets sous différents conditions lumineuses
- Le dataset Glass contient les données de composition de verres ainsi que l'indice de réfraction
- Le dataset Ionos contient des données radars passant par la ionosphere

# aloi
Meilleurs paramètres:
- {'contamination': 0.05, 'n_neighbors': 5}
- Meilleur F1 score : 0.26
- Précision : 94.01%
- Recall : 33.75%
- Donnée du fournisseur: ALOI 49534 objects, 1508 outliers (3.04%) ALOI

# glass
- Meilleurs paramètres:
- {'contamination': 0.05, 'n_neighbors': 5}
- Meilleur F1 score : 0.20
- Précision : 95.79%
- Recall : 11.11%
- Donnée du fournisseur: 7 attributes, 214 objects, 9 outliers (4.21%)

# ionos
- Meilleurs paramètres:
- {'contamination': 0.1, 'n_neighbors': 3}
- Meilleur F1 score : 0.31
- Précision : 68.38%
- Recall : 19.84%
- Donnée du fournisseur: 32 attributes, 351 objects, 126 outliers (35.90%)
